In [7]:
import pandas as pd
df1 = pd.read_csv('act_commuters.csv')
df2 = pd.read_csv('act_non_commuter.csv')
df = pd.concat([df1,df2],axis = 0)

In [5]:
df2['state_sequence']

0        ['home', 'home', 'home', 'home', 'home', 'home...
1        ['home', 'home', 'home', 'home', 'home', 'home...
2        ['home', 'home', 'home', 'home', 'home', 'home...
3        ['home', 'home', 'home', 'home', 'home', 'home...
4        ['home', 'home', 'home', 'home', 'home', 'home...
                               ...                        
12389    ['home', 'home', 'home', 'home', 'home', 'home...
12390    ['home', 'home', 'home', 'home', 'home', 'home...
12391    ['home', 'home', 'home', 'home', 'home', 'home...
12392    ['home', 'home', 'home', 'home', 'home', 'home...
12393    ['home', 'home', 'home', 'home', 'home', 'home...
Name: state_sequence, Length: 12394, dtype: object

In [14]:
df = df.drop_duplicates(subset='person', keep='first')

In [16]:
df['state_sequence'].dropna()
df

,Unnamed: 0,person,purpose,arr_time,dep_time,gender,education,driver_license,X_transit_pass,employment,...,Nw,num_states,duration,states,beta1,beta2,state_sequence,num_of_trips,arr_time_work,dep_time_work
0,0,10420111,3,35,50,1,3,1,0,1,...,1.0,4,15,"[[35, 15], [71, 0]]",7,121,"['home', 'home', 'home', 'home', 'home', 'home...",3,72.0,96.0
2,2,10436131,15,54,55,2,5,1,0,1,...,2.0,5,1,"[[54, 1], [70, 14]]",3,11,"['home', 'home', 'home', 'home', 'home', 'home...",2,35.0,67.0
4,4,10607491,25,24,24,2,5,0,1,1,...,1.0,3,0,"[[24, 0]]",7,1,"['home', 'home', 'home', 'home', 'home', 'home...",2,26.0,60.0
5,5,10646441,19,84,86,1,4,1,0,1,...,2.0,4,2,"[[84, 2]]",3,131,"['home', 'home', 'home', 'home', 'home', 'home...",2,33.0,66.0
6,6,10646442,25,80,102,1,2,1,0,1,...,0.0,2,22,"[[80, 22]]",7,21,"['home', 'home', 'home', 'home', 'home', 'home...",0,37.0,78.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12386,12386,72114261,17,41,42,1,6,1,0,0,...,1.0,3,1,"[[41, 1]]",9,91,"['home', 'home', 'home', 'home', 'home', 'home...",4,NaN,NaN
12387,12387,72115601,15,52,53,2,5,1,0,0,...,2.0,5,1,"[[52, 1], [79, 1]]",4,141,"['home', 'home', 'home', 'home', 'home', 'home...",14,NaN,NaN
12389,12389,72116692,21,42,46,2,5,1,0,0,...,1.0,3,4,"[[42, 4]]",10,11,"['home', 'home', 'home', 'home', 'home', 'home...",0,NaN,NaN
12390,12390,72117985,25,12,48,1,1,0,0,0,...,0.0,3,36,"[[12, 36], [50, 6], [61, 46]]",8,141,"['home', 'home', 'home', 'home', 'home', 'home...",0,NaN,NaN


In [19]:
activity = pd.DataFrame(columns=['id', 'perno', 'activity_type', 'start_time', 'end_time', 'purpose'])

for index, row in df.iterrows():
    state_sequence = eval(row['state_sequence'])
    current_activity = state_sequence[0]
    start_time = 0
    actno = 1
    for time, state in enumerate(state_sequence):
        if state == 'other1':  # start of a new 'other' activity
            state = 'other'
            activity_change = True
        else:
            activity_change = state != current_activity

        if activity_change or time == len(state_sequence) - 1:  # activity changed or end of day
            end_time = time
            purpose = 1 if current_activity == 'home' else 2 if current_activity == 'work' else 3
            new_row = pd.DataFrame({
                'id': [row['person']], 
                'actno': [actno],
                'activity_type': [current_activity], 
                'start_time': [start_time], 
                'end_time': [end_time], 
                'purpose': [purpose]
            })
            activity = activity.append(new_row, ignore_index=True)
            start_time = time
            current_activity = state
            if activity_change:
                actno += 1

C:\Users\15123\AppData\Local\Temp\ipykernel_20828\1052161617.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  activity = activity.append(new_row, ignore_index=True)


In [20]:
activity

,id,perno,activity_type,start_time,end_time,purpose,actno
0,10420111,NaN,home,0,60,1,1.0
1,10420111,NaN,other,60,64,3,2.0
2,10420111,NaN,other,64,81,3,3.0
3,10420111,NaN,work,81,97,2,4.0
4,10420111,NaN,home,97,107,1,5.0
...,...,...,...,...,...,...,...
36755,72119121,NaN,other,63,64,3,8.0
36756,72119121,NaN,other,64,65,3,9.0
36757,72119121,NaN,other,65,66,3,10.0
36758,72119121,NaN,other,66,76,3,11.0


In [23]:
dist = pd.read_csv('df_train_ATM.csv')
dist = dist[dist['purpose'] !=1]
dist = dist[dist['purpose'] !=2]

In [28]:
dist_purpose = dist['purpose'].value_counts(normalize=True)
act = activity.copy()
activity.to_csv('1.csv')

In [30]:
import numpy as np
def replace_purpose(row):
    if row['purpose'] == 3:
        return np.random.choice(dist_purpose.index, p=dist_purpose.values)
    else:
        return row['purpose']

activity['purpose'] = activity.apply(replace_purpose, axis=1)

In [31]:
activity

,id,perno,activity_type,start_time,end_time,purpose,actno
0,10420111,NaN,home,0,60,1,1.0
1,10420111,NaN,other,60,64,17,2.0
2,10420111,NaN,other,64,81,21,3.0
3,10420111,NaN,work,81,97,2,4.0
4,10420111,NaN,home,97,107,1,5.0
...,...,...,...,...,...,...,...
36755,72119121,NaN,other,63,64,3,8.0
36756,72119121,NaN,other,64,65,17,9.0
36757,72119121,NaN,other,65,66,19,10.0
36758,72119121,NaN,other,66,76,18,11.0


In [44]:
def generate_list(row):
    return list(row[['purpose', 'start_time', 'end_time']].values)

new_df = activity.groupby('id').apply(generate_list).reset_index()
new_df.columns = ['id', 'activity']

In [49]:
new_df.to_csv('timegeo.csv', index=False, header=True)

In [48]:
new_df['activity'] = new_df['activity'].apply(lambda x: [i.tolist() for i in x])
new_df['activity'] = new_df['activity'].apply(str)
new_df.to_csv('timegeo.csv', index=False, header=True)

AttributeError: 'str' object has no attribute 'tolist'

In [45]:
new_df

,id,activity
0,10319851,"[[1, 0, 107]]"
1,10319852,"[[1, 0, 83], [20, 83, 85], [1, 85, 107]]"
2,10320364,"[[1, 0, 80], [15, 80, 83], [1, 83, 107]]"
3,10320531,"[[1, 0, 46], [24, 46, 48], [24, 48, 50], [21, ..."
4,10320534,"[[1, 0, 107]]"
...,...,...
8647,72115601,"[[1, 0, 31], [19, 31, 32], [19, 32, 47], [24, ..."
8648,72115602,"[[1, 0, 29], [2, 29, 39], [1, 39, 52], [12, 52..."
8649,72116692,"[[1, 0, 107]]"
8650,72117985,"[[1, 0, 107]]"
